# Install

In [24]:
# # Basis
# %pip install --upgrade pip --quiet

# # NLP
# %pip install pandas --quiet
# %pip install spacy --quiet
# # 效率
# # !python -m spacy download en_core_web_md
# # !python -m spacy download en_core_web_sm
# # 準確性 
# !spacy download en_core_web_trf -q

# # NLP 2
# # !pip install nltk

# # For Map
# %pip install geopy --quiet

# # other
# %pip install pandas

# Import

In [25]:
from pprint import pprint
import spacy
nlp = spacy.load("en_core_web_trf");
from geopy.geocoders import Nominatim
# import pandas as pd
import json
import os

/Users/ansoncar/Sync/My/MyProject/NLP_in_map/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_di

## exsample

In [60]:

doc = nlp("Welcome to hong kong")
print(doc)
print([(w.text, w.pos_) for w in doc])

for ent in doc.ents:
    if ent.label_ == 'GPE':  # GPE 表示地理政治实体
        print(ent.text, ent.label_)

Welcome to hong kong
[('Welcome', 'INTJ'), ('to', 'ADP'), ('hong', 'PROPN'), ('kong', 'PROPN')]
hong kong GPE


# 1. 預處理數據

## 根據 [ 章節號碼 ] 分出 十八章節，存放在 chapter list 之中

In [71]:
chapters = []
chapter_num = 0

file_path = './data/JamesJoyce-Ulysses_Ans.txt'

# open file
# with open(file_path, "r", encoding='utf-8-sig') as file:
with open(file_path, "r") as file:
    # loop each line
    for line in file:
        # 分章節
        if line.strip().startswith(f'[ { chapter_num+1} ]'):
            chapter_num += 1
            chapters.append('')
            # 唔要章節號碼
        else:
            # save line
            chapters[chapter_num-1] += line

# 根據 \n\n，幫每一個章節 分每一句句子出泥

In [72]:
chapters_v2 = []

for chapter in chapters:
   chapters_v2.append(chapter.split('\n\n'))

# 根據 \n，幫每一句句子嘅\n 變成 空格，合返做一行

In [73]:
chapters_v3 = []
# 每一個章節
for chapter_num, chapter in enumerate(chapters_v2):
    chapters_v3.append([])
    # 每一個句子
    for index, line  in enumerate(chapter):
        # 換 \n 同 去頭尾空格
        text = str(line).replace('\n', ' ').strip()
        # 去除空白行
        if (text != '' and text != ' '):
            chapters_v3[chapter_num].append(text)

# 2. NLP Time

# check 每一句句子嘅地點

In [74]:
# for chapter in chapters_v3:
#     for line in chapter:
#         spacy.displacy.render(nlp(line),style="ent")

doc = nlp(chapters_v3[0][0])
print(doc)
print([(w.text, w.pos_) for w in doc])

spacy.displacy.render(doc,style="ent")

Stately, plump Buck Mulligan came from the stairhead, bearing a bowl of lather on which a mirror and a razor lay crossed. A yellow dressinggown, ungirdled, was sustained gently behind him on the mild morning air. He held the bowl aloft and intoned:
[('Stately', 'ADV'), (',', 'PUNCT'), ('plump', 'ADJ'), ('Buck', 'PROPN'), ('Mulligan', 'PROPN'), ('came', 'VERB'), ('from', 'ADP'), ('the', 'DET'), ('stairhead', 'NOUN'), (',', 'PUNCT'), ('bearing', 'VERB'), ('a', 'DET'), ('bowl', 'NOUN'), ('of', 'ADP'), ('lather', 'NOUN'), ('on', 'ADP'), ('which', 'PRON'), ('a', 'DET'), ('mirror', 'NOUN'), ('and', 'CCONJ'), ('a', 'DET'), ('razor', 'NOUN'), ('lay', 'VERB'), ('crossed', 'VERB'), ('.', 'PUNCT'), ('A', 'DET'), ('yellow', 'ADJ'), ('dressinggown', 'NOUN'), (',', 'PUNCT'), ('ungirdled', 'VERB'), (',', 'PUNCT'), ('was', 'AUX'), ('sustained', 'VERB'), ('gently', 'ADV'), ('behind', 'ADP'), ('him', 'PRON'), ('on', 'ADP'), ('the', 'DET'), ('mild', 'ADJ'), ('morning', 'NOUN'), ('air', 'NOUN'), ('.', 'PU

# 實做

In [75]:
# gpe_data = []
# # 每一個章節
# for chapter_num, chapter in enumerate(chapters_v3):
#     gpe_data.append({})
#     # 每一句句子
#     for line_num, line in enumerate(chapter):
#         text = nlp(line)
#         for ent in text.ents:
#             if ent.label_ == "GPE":  # GPE 表示地理政治实体
#                 gpe_name = gpe_data[chapter_num].get(ent.text)
#                 if not gpe_name:
#                     gpe_data[chapter_num][ent.text] = 0
#                 gpe_data[chapter_num][ent.text] += 1
#     break
# gpe_data

# 3. 揾地點 座標

In [76]:
geolocator = Nominatim(user_agent="damn boy")
location = geolocator.geocode("New York City")

if location:
    print((location.latitude, location.longitude))
else:
    print("Location not found")

(40.7127281, -74.0060152)


# 寫一個 function for loop 曬所有 chapter

In [99]:
def get_gpe(chapter: list):
    geolocator = Nominatim(user_agent="damn boy")
    ret = {}
    # 每一句句子
    for line_num, line in enumerate(chapter):
        # 分析句子
        text = nlp(line)
        # 查看每一個字詞性
        for ent in text.ents:
            # GPE 表示地理政治实体，係就拎出泥
            if ent.label_ == "GPE":
                if not ret.get(ent.text):
                    ret[ent.text] = {"nominatim": [], "lines": [], "count": 0, 'text':[]}
                    query = f"{ent.text}, Ireland"
                    location = geolocator.geocode(query)
                    if location:
                        ret[ent.text]["nominatim"] = [location.latitude, location.longitude]
                    else:
                        print(ent.text, "Location not found")

                if line_num not in ret[ent.text]["lines"] and line not in ret[ent.text]["text"]:
                    ret[ent.text]["lines"].append(line_num)
                    ret[ent.text]["text"].append(line)
                    ret[ent.text]["count"] += 1
    return ret

# output json file
def toJson(data:dict, name:str):
    json_output = json.dumps(data, indent=4, ensure_ascii=False)
    output_dir = './dist/'

    # 檢查目標目錄是否存在，不存在則創建
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(f'{output_dir}{name}.json', 'w', encoding='utf-8' ) as f:
        f.write(json_output)

# 開始所有

In [97]:
# for chapter_num, chapter in enumerate(chapters_v3):
#     print(f'RUN chapter {chapter_num+1}')
#     data = get_gpe(chapter)
#     toJson(data, f'chapter_{chapter_num+1}_gpe')

---

In [100]:
num = 1
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 1
Dottyville Location not found


In [101]:
num = 2
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 2
Mürzsteg Location not found


In [102]:
num = 3
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 3
Barcelona Location not found
Romeville Location not found


In [103]:
num = 4
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 4
Kinnereth Location not found
Tiberias Location not found
Sodom Location not found
Gomorrah Location not found
Edom Location not found


In [104]:
num = 5
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 5


In [105]:
num = 6
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 6
the Isle of Man Location not found
Roundtown Location not found


In [106]:
num = 7
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 7
Clonskea Location not found
Manx Location not found
Brixton Location not found
SHINDY Location not found
Habsburg Location not found
Roundtown Location not found
VIRGILIAN Location not found
SOPHOMORE Location not found
FLO WANGLES Location not found


In [107]:
num = 8
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 8
Zion Location not found
exc. Location not found
Rathoath Location not found
Kerwan Location not found
the Red Bank Location not found
Margate Location not found
Levenston Location not found


In [108]:
num = 9
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 9
Mallarmé Location not found
Pickwick Location not found
Romeville Location not found
Clamart Location not found
Sheba Location not found
Almany Location not found
Ultonian Antrim Location not found
Wittenberg Location not found
Southwark Location not found
Cymbeline Location not found


In [109]:
num = 10
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 10
Fehrenbach Location not found
Potterton Location not found
palmam ferenti Location not found
Llandudno Location not found
Bridgwater Location not found


In [110]:
num = 11
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 11
Yessex Location not found
Ternoon Location not found
Dedalus Location not found
Yashmak Location not found
Persia Location not found


In [111]:
num = 12
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 12
Exeter Location not found
Iffley Location not found
Playwood Location not found
Stoke Newington Location not found
Chepstow Location not found
the United Kingdom Location not found
O’Dignam Location not found
Pentonville Location not found
Buncombe Location not found
Heligoland Location not found
Oakholme Regis Location not found
Omaha Location not found
Rio de Janeiro Location not found
Tetuan Location not found
Carrantuohill Location not found
Clonmacnois Location not found
Lough Neagh’s Location not found
Castleconnel Location not found
Kilballymacshonakill Location not found
Abeakuta Location not found
Cottonopolis Location not found
the Congo Free State Location not found
Viterbo Location not found


In [112]:
num = 13
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 13
Borneo Location not found


In [113]:
num = 14
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 14
Alba Longa Location not found
Almany Location not found
Madagascar Location not found
Oxtail Location not found
Horeb Location not found
Pisgah Location not found
the Horns of Hatten Location not found
Tophet Location not found
Cape Horn Location not found
Bashan Location not found
Lethe Location not found
linseywoolsey Location not found
Juda Location not found
Chawley Location not found
Au reservoir Location not found
mossoo Location not found
chokeechokee Location not found
Bawdyhouse Location not found
Vladivostok Location not found


In [114]:
num = 15
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 15
Zion Location not found
Livermore Location not found
Bloemfontein Location not found
LONGHAND Location not found
SHORTHAND Location not found
Kinnereth Location not found
the North Riding of Tipperary Location not found
Bloomusalem Location not found
the Nova Hibernia Location not found
PAPAL NUNCIO Location not found
Agendath Netaim Location not found
Mizraim Location not found
CITRON Location not found
Istria Location not found
Pflaap Location not found
Nankeen Location not found
Szombathely Location not found
Candia Location not found
Watercloset Location not found


In [115]:
num = 16
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 16
Stockholm Location not found
Bridgwater Location not found
Beni Location not found
Bolivia Location not found
Santiago Location not found
Southampton Location not found
Margate Location not found
Scarborough Location not found
Bournemouth Location not found
Swansea Location not found
Ramhead Location not found


In [116]:
num = 17
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 17
Roundtown Location not found
Zion Location not found
Szombathely Location not found
jehudi Location not found
homijah Location not found
Calcata Location not found
Mizrach Location not found
Southwark Location not found
Flowerville Location not found
covin Location not found
venville Location not found
Midlothian Location not found
Ripon Location not found
Szesfehervar Location not found
Florence Location not found
La Linea Location not found
Thibet Location not found


In [117]:
num = 18
print(f'RUN chapter {num}')
data = get_gpe(chapters_v3[num-1])
toJson(data, f'chapter_{num}_gpe')

RUN chapter 18
Bloemfontein Location not found
Algeciras Location not found
Lewers Location not found
Helys Location not found
Inces Location not found
Trilby Location not found
Lahore Location not found
Abrines Location not found
Liptons Location not found
Trieste-Zurich Location not found
Salt Lake City Location not found


---

In [95]:
def clean_text_line(line: str, target_str: str) -> str:
    """
    Shorten the text line by trimming the line by 
    footstop (.), question mark (?) or exclamation mark (!),
    getting only the text around the target string

    Args:
        line (str): The text line to be cleaned
        target_str (str): The target string

    Returns:
        str: The cleaned text line
    """
    # Find the target string in the line
    target_index = line.find(target_str)
    # Find the first footstop (.), question mark (?) or exclamation mark (!) after the target string
    footstop_index = line.find(".", target_index)
    question_index = line.find("?", target_index)
    exclamation_index = line.find("!", target_index)
    # Get the shortest index
    if footstop_index == -1:
        footstop_index = len(line)
    if question_index == -1:
        question_index = len(line)
    if exclamation_index == -1:
        exclamation_index = len(line)
    # Get the shortest index
    end_index = min(footstop_index, question_index, exclamation_index)
    # Get the last footstop (.), question mark (?) or exclamation mark (!) before the target string
    footstop_index = line.rfind(".", 0, target_index)
    question_index = line.rfind("?", 0, target_index)
    exclamation_index = line.rfind("!", 0, target_index)
    # Get the longest index
    start_index = max(footstop_index, question_index, exclamation_index)
    # Return the cleaned text line
    return line[start_index+1:end_index]
    

def filter_locations(latitude: float, longitude: float) -> bool:
    """
    Filter the locations based on the latitude and longitude

    Args:
        latitude (float): The latitude of the location
        longitude (float): The longitude of the location

    Returns:
        bool: True if the location is within the range, False otherwise
    """
    # Top left: 55.63588022503673, -10.958074311157244
    # Bottom right: 51.10420379246764, -5.0255379638926465
    if latitude < 51.10420379246764 or latitude > 55.63588022503673:
        return False
    if longitude < -10.958074311157244 or longitude > -5.0255379638926465:
        return False
    return True


In [98]:
chapter_list = []

for i in range(0, 18):
    chapter = get_gpe(chapters_v3[i])
    for item in chapter:
        # Skip if the nominatim is empty
        if not chapter[item]["nominatim"]:
            continue
        if filter_locations(chapter[item]["nominatim"][0], chapter[item]["nominatim"][1]):
            chapter_list.append({
                "chapter": i+1,
                "gpe": item,
                "nominatim": chapter[item]["nominatim"],
                "count": chapter[item]["count"],
                "part_of_text": clean_text_line(chapter[item]["part_of_text"], item)
            })

Dottyville Location not found


In [99]:
for item in chapter_list:
    for gpe in item:
        pprint(item[gpe])
        print()

1

'Dublin'

[53.3493795, -6.2605593]

1

('He mounted to the parapet again and gazed out over Dublin bay, his fair '
 'oakpale hair stirring slightly')

1

'Kingstown'

[53.2922794, -6.1360079]

1

(' Leaning on it he looked down on the water and on the mailboat clearing the '
 'harbourmouth of Kingstown')

1

'Bray Head'

[53.1904724, -6.0837452]

1

('They halted, looking towards the blunt cape of Bray Head that lay on the '
 'water like the snout of a sleeping whale')

1

'Haines'

[53.2894887, -6.1297938]

4

' Haines is apologising for waking us last night'

1

'Clongowes'

[53.310525299999995, -6.686760774260252]

1

' So I carried the boat of incense then at Clongowes'

1

'Dundrum'

[53.2891457, -6.2433756]

1

(' Five lines of text and ten pages of notes about the folk and the fishgods '
 'of Dundrum')

1

'Ireland'

[53.429500000000004, -8.1227484875]

3

('—He’s English, Buck Mulligan said, and he thinks we ought to speak Irish in '
 'Ireland')

1

'Elsinore'

[51.92629515,

---